In [1]:
import pymapee as pymap
import geemap as emap
import ee
pymap.initialize_ee()

# 1. Masking cloud and produce monthly composites

In [2]:
# MODIS NDVI Terra dataset
ndvi=ee.ImageCollection("MODIS/061/MOD13A2")

In [3]:
# Mask cloud-related pixels and slect NDVI band
modis_mask=pymap.modis_cloud_mask(ndvi, 2, 5, "DetailedQA", 1).select("NDVI")

In [4]:
# Make monthly NDVI composite
monthly_composite=pymap.monthly_composite(modis_mask, mode="median").select("NDVI")

In [5]:
# Scaling NDVI data
composite_scale=pymap.scaling_data(monthly_composite, 0.0001).filterDate("2015-01-01","2020-01-01")

In [6]:
# If you don't have geemap package, please install it (!pip install geemap).
Map=emap.Map()
vis_params = {'bands': ['NDVI'], 'palette': ['9e0142', ' d0384e', ' ee6445', ' fb9c59', ' fece7c', ' fff1a7', ' f4faad', ' d1ec9c', ' 98d6a4', ' 5db7a9', ' 3683bb', ' 5e4fa2'], 'min': 0.0, 'max': 1}
Map.addLayer(composite_scale.first(),vis_params, "Monthly composite")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# 2. Calculating Vegetation anomaly and condition indices

In [7]:
# Vegetation anomaly index (VAI)
vai=pymap.VAI(composite_scale)

In [8]:
Map=emap.Map()
vis_params = {'bands': ['VAI'], 'palette': ['9e0142', ' d0384e', ' ee6445', ' fb9c59', ' fece7c', ' fff1a7', ' f4faad', ' d1ec9c', ' 98d6a4', ' 5db7a9', ' 3683bb', ' 5e4fa2'], 'min': -0.1, 'max': 0.5}
Map.addLayer(vai.first(),vis_params, "Monthly composite")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [9]:
# Vegetation condition index (VCI)
vci=pymap.VCI(composite_scale)

In [10]:
Map=emap.Map()
vis_params = {'bands': ['VCI'], 'palette': ['9e0142', ' d0384e', ' ee6445', ' fb9c59', ' fece7c', ' fff1a7', ' f4faad', ' d1ec9c', ' 98d6a4', ' 5db7a9', ' 3683bb', ' 5e4fa2'], 'min': 1, 'max': 100}
Map.addLayer(vci.first(),vis_params, "Monthly composite")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# 3. Creating daily composite

In [11]:
# ERA5-Land Hourly datasets
era5=ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select("temperature_2m").filterDate("2000-01-01","2020-12-31")
# The total number of hourly images from the collection
print(f"Number of hourly images: {era5.size().getInfo()}")

Number of hourly images: 184080


In [15]:
daily_composite=pymap.daily_composite(era5)
# The total number of daily images (hours) from the collection
print(f"Number of daily images: {daily_composite.size().getInfo()}")

Number of daily images: 7670


# 4. Download NDVI time-series data given the area of interest.

In [12]:
aoi = ee.FeatureCollection("projects/tuyenhavan/assets/Vietnam_PhD")

In [13]:
pymap.export_to_googledrive(monthly_composite, aoi, "Export_Data","Monthly_dataset", res=1000)

When downloading data over a large area, the downloaded data may be split into various tiles. You can merge them all together.